In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pyabf
import os
import pandas as pd


In [46]:
file = r"C:\Users\dongq\OneDrive\mouse patch-seq\Mecp2 project\3 weeks\2023_05_05_0001.abf"

In [36]:
df = pd.DataFrame({'File': ['2023_04_10_0003.abf', '2023_04_10_0004.abf', '2023_04_10_0005.abf']})

In [40]:
df

,File
0,2023_04_10_0003.abf
1,2023_04_10_0004.abf
2,2023_04_10_0005.abf


In [47]:
f = pyabf.ABF(file)


In [4]:
from feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor
sfe = SpikeFeatureExtractor(filter=2)
spte = SpikeTrainFeatureExtractor(start=0, end=16000)


In [48]:
for index in f.sweepList:
    f.setSweep(index)
    print(f.sweepEpochs.levels, f.sweepEpochs.types)

[0.0, 0.0, -100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 0.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 120.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 140.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 160.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 180.0, 0.0] ['Step', 'Step', 'Step', 'Step']


In [41]:


sampling_rate = f.sampleRate
temp_result_list = []
flag_spike = False
for index in f.sweepList:
    f.setSweep(index)
    t = f.sweepX
    v = f.sweepY
    i = f.sweepC

    if f.sweepUnitsY == f.sweepUnitsC:
        v = v/20
    ft = sfe.process(t, v, i)
    # ft.to_csv(f'{index}.csv', index=False)
    sptft= spte.process(t=t, v=v, i=i, spikes_df=ft)
    current = f.sweepEpochs.levels[1]
    sptft['current'] = current
    temp_result_list.append((ft, sptft))

for i_result in temp_result_list:
    if i_result[1]['avg_rate'] > 0:
        new_df = i_result[0].mean().to_frame().T
        new_df.insert(0, 'File', os.path.basename(file))
        break
for i_result in temp_result_list:
    if 'adapt' in i_result[1] and i_result[1]['adapt'] > 0:
        temp_dict = {i: [i_result[1][i]] for i in i_result[1]} 
        fire_pattern_df = pd.DataFrame(temp_dict)
        fire_pattern_df.insert(0, 'File', os.path.basename(file))
        break
firing_rate_list = dict()
firing_rate_list['File'] = [os.path.basename(file)]
for i_result in temp_result_list:
    current = i_result[1]['current']
    rate = i_result[1]['avg_rate'] * sampling_rate
    firing_rate_list[f'{current} pA'] = [rate]
rate_df = pd.DataFrame(firing_rate_list)
ccc = pd.concat([df, pd.merge(new_df, pd.merge(fire_pattern_df, rate_df, on='File'), on='File')])
ccc.to_csv('test.csv', index=False)
    


C:\Users\dongq\AppData\Local\Temp\ipykernel_28212\102314537.py:21: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_df = i_result[0].mean().to_frame().T


In [42]:
ccc

,File,threshold_index,clipped,threshold_t,threshold_v,threshold_i,peak_index,peak_t,peak_v,peak_i,...,15.0 pA,20.0 pA,25.0 pA,30.0 pA,35.0 pA,40.0 pA,45.0 pA,50.0 pA,55.0 pA,60.0 pA
0,2023_04_10_0003.abf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023_04_10_0004.abf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023_04_10_0005.abf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023_04_10_0003.abf,6478.0,0.0,0.6478,-46.508789,5.0,6515.0,0.6515,2.258301,5.0,...,2.5,3.125,3.75,3.75,3.75,4.375,1.875,1.25,1.25,0.625
